# Transfer Learning in NMT with attention

## Load Datasets

Telugu English Parallel corpus

In [1]:
en_te_train = open('./data/en-te/en-te/train.en').readlines()[:100]
te_train = open('./data/en-te/en-te/train.te', encoding='utf-8').readlines()[:100]

for pair in list(zip(te_train[:5], en_te_train[:5])):
    print(f'{pair[0]}\t{pair[1]}')

మళ్లీ ఉదయిస్తాడు.
	Rise again.

యెహోవా కృపను మనమెలా మహిమపరచవచ్చు?
	How do we glorify Jehovahs undeserved kindness?

ఆర్థికంగా కూడా భారత్‌ వే గంగా పయనిస్తున్నది.
	India also continues to push back economically.

‘విద్యార్థులను చూస్తుంటే నాకు చిన్నప్పటి రోజులు గుర్తుకొస్తున్నాయి.
	I remember my childhood days.

ఆర్థిక లావాదేవీలన్నీ ఆన్‌లైన్‌లోనే
	All transactions are made online.



Kannada English Parallel corpus

In [2]:
en_kn_train = open('./data/en-kn/en-kn/train.en').readlines()
kn_train = open('./data/en-kn/en-kn/train.kn', encoding='utf-8').readlines()

for pair in list(zip(kn_train[:5], en_kn_train[:5])):
    print(f'{pair[0]}\t{pair[1]}')

ಸಾಮಾಜಿಕ ಜಾಲತಾಣಗಳಲ್ಲಿ ಅದೆಷ್ಟೊ ವಿಡಿಯೋಗಳು ಹರಿದಾಡುತ್ತಿರುತ್ತವೆ.
	Such videos are aplenty on social media channels.

ಮುಳಗುಂದ ಪೊಲೀಸ್ ಠಾಣಾ ವ್ಯಾಪ್ತಿಯಲ್ಲಿ ಈ ದುರ್ಘಟನೆ ನಡೆದಿದೆ…
	This incident happened within the limits of Mudhol Police Station.

ಪ್ರಸ್ತುತ ಪರಿಸ್ಥಿತಿ ನಿಯಂತ್ರಣದಲ್ಲಿದೆ ಎಂದು ತಿಳಿಸಿದ್ದಾರೆ.
	The situation is currently under control, he said.

ನಿಮ್ಮ ನೆಲೆ ಪುಟವಾಗಿಸಲು ಒಂದು ಬುಕ್‌ಮಾರ್ಕ್ ಅನ್ನು ಆರಿಸಿ. ನೀವು ಒಂದು ಕೋಶವನ್ನು ಆರಿಸಿದರೆ, ಅದರಲ್ಲಿನ ಬುಕ್‌ಮಾರ್ಕುಗಳು ಪ್ರತ್ಯೇಕ ಹಾಳೆಗಳಾಗಿ ತೆರೆಯಲ್ಪಡುತ್ತವೆ.
	Choose a Bookmark to be your Home Page. If you choose a folder, the Bookmarks in that folder will be opened in Tabs.

ಇದಕ್ಕೆ ಸಾಮಾಜಿಕವಾದ ಹಲವು ವ್ಯವಸ್ಥೆಗಳು ಪೂರಕವಾಗಿ ಕಾರ್ಯ ನಿರ್ವಹಿಸುತ್ತವೆ.
	Several social organisations are also being involved.



In [3]:
len(kn_train), len(te_train), len(en_kn_train), len(en_te_train)

(4014931, 100, 4014931, 100)

## NMT



In [4]:
!pip install keras

In [5]:
from tensorflow import keras
import numpy as np

min_samples = 10000
batch_size = 64
epochs = 100
latent_dim = 256

In [6]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# english tokens
all_en = en_te_train
#all_en.extend(en_kn_train)

for line in all_en:
    for char in line:
        if char not in input_characters:
            input_characters.add(char)

# telugu tokens
for line in te_train[:min(min_samples, len(te_train) - 1)]:
    for char in line:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in all_en])
max_decoder_seq_length = max([len(txt) for txt in te_train])

In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(all_en), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(all_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(all_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(all_en, te_train)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

## Building the model

In [8]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Training the model

In [9]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=10,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/10
2/2 [==============================] - 16s 3s/step - loss: 4.3261 - accuracy: 0.1894 - val_loss: 0.5345 - val_accuracy: 0.9340
Epoch 2/10
2/2 [==============================] - 10s 2s/step - loss: 0.6072 - accuracy: 0.9248 - val_loss: 0.4547 - val_accuracy: 0.9308
Epoch 3/10
2/2 [==============================] - 11s 2s/step - loss: 0.5077 - accuracy: 0.9220 - val_loss: 0.4348 - val_accuracy: 0.9324
Epoch 4/10
2/2 [==============================] - 10s 2s/step - loss: 0.4767 - accuracy: 0.9231 - val_loss: 0.4145 - val_accuracy: 0.9309
Epoch 5/10
2/2 [==============================] - 11s 2s/step - loss: 0.4558 - accuracy: 0.9218 - val_loss: 0.3959 - val_accuracy: 0.9326
Epoch 6/10
2/2 [==============================] - 12s 3s/step - loss: 0.4738 - accuracy: 0.9235 - val_loss: 1.9149 - val_accuracy: 0.9328
Epoch 7/10
2/2 [==============================] - 11s 2s/step - loss: 1.6350 - accuracy: 0.9233 - val_loss: 0.4377 - val_accuracy: 0.9286
Epoch 8/10
2/2 [==================

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


## Sampling

In [23]:
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [34]:
decoder_model.predict(encoder_input_data)

ValueError: in user code:

    C:\Users\gauth\Anaconda\lib\site-packages\keras\engine\training.py:1586 predict_function  *
        return step_function(self, iterator)
    C:\Users\gauth\Anaconda\lib\site-packages\keras\engine\training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\gauth\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\gauth\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\gauth\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\gauth\Anaconda\lib\site-packages\keras\engine\training.py:1569 run_step  **
        outputs = model.predict_step(data)
    C:\Users\